### 데이터 확인 및 불러오기

- 데이터는 구글 드라이브 > 프로젝트 > 중급 프로젝트 > 원본데이터에 있어요.
- 제공된 문서 파일들과 `data_list.csv` 파일에 담긴 메타데이터를 확인합니다.
- 문서 파일을 불러옵니다. `hwp`와 `pdf` 두 가지 포맷에 대응할 수 있어야 해요.
- 유용한 메타데이터가 무엇일지 판단하여 함께 활용해 보세요.

### 문서 청킹

- 청크 크기, 그리고 청크 간 중첩 크기를 잘 설정하여 문서를 청킹합니다.
- (심화) 제안서 포맷을 활용해 의미 단위까지 고려하여 문서를 청킹해 보고 성능을 확인해 보세요.

### 임베딩 생성

- 임베딩 성능과 비용을 고려해 적절한 임베딩 모델을 선정합니다.
- 사용의 편의성과 검색 기능을 고려해 적절한 Vector DB를 선정합니다.

### Retrieval 기능 구현

- 먼저 naive한 retrieval을 구현해 베이스라인으로 삼으면 좋습니다. 베이스라인을 기반으로 점점 더 retrieval 기능을 고도화해 나가면 됩니다.
- 데이터 소스 문서가 하나가 아니라 여러 개이기 때문에, 타깃 문서를 정확히 찾기 위해 메타데이터 필터링을 활용할 수 있습니다. 이때 사용자가 발주 기관이나 사업명 같은 정보를 비슷하지만 정확하지는 않게 입력하는 케이스도 고려하는 게 좋아요.
- (심화) Retrieval과 관련해 다양한 옵션과 기법을 실험하면서 성능을 확인해 보세요.
    - Retrieval을 위한 프롬프트 엔지니어링
    - Top-k 검색에서 k 값 설정
    - 단순 유사도 기반 검색 / MMR(Maximum Marginal Relevance) / Hybrid Search
    - Multi-Query, Re-Ranking 등 심화 기법

### Generation 기능 구현

- 텍스트 생성 능력과 비용을 고려해 적절한 언어 모델을 선정합니다.
- 원하는 답변 양상과 분량에 따라서 temperature, top_p, max tokens 등 텍스트 생성과 관련된 옵션도 적절하게 설정해 주세요.
- 답변 생성을 위한 최적의 프롬프트를 작성합니다.
    - Retrieval 과정에서 찾은 컨텍스트를 충실히 반영해야 합니다.
    - 불필요한 내용이 답변에 포함되지 않도록 합니다.
    - 응답의 톤이나 스타일을 조정합니다.
    - 비용을 고려해 토큰 사용량도 최적화해 보세요.
- RAG 시스템이 대화 맥락을 유지하여 사용자와 대화를 이어 갈 수 있도록 해 주세요.

### 성능 평가

- 답변의 품질을 평가할 때는 다음과 같은 기준을 사용할 수 있습니다.
    - 사용자가 요청한 내용을 단일 문서에서 정확하게 뽑아내 답변하는지
    - 사용자가 여러 문서에 대해 요청한 내용을 잘 종합해서 답변하는지
    - 후속 질문의 맥락을 잘 이해하고 답변하는지
    - 문서에 포함되어 있지 않은 내용에 대해서는 모른다고 답변하는지
- 위와 같은 기준을 어떤 방식으로 평가하고 어떤 지표로 나타낼지 선정해 보세요.
- 주어진 문서 데이터에 맞게 다양한 질문 세트를 준비하여 성능 평가에 활용해 보세요.
    - 질문 예시
        - 국민연금공단이 발주한 이러닝시스템 관련 사업 요구사항을 정리해 줘.
            - 콘텐츠 개발 관리 요구 사항에 대해서 더 자세히 알려 줘.
            - 교육이나 학습 관련해서 다른 기관이 발주한 사업은 없나?
        - 기초과학연구원 극저온시스템 사업 요구에서 AI 기반 예측에 대한 요구사항이 있나?
            - 그럼 모니터링 업무에 대한 요청사항이 있는지 찾아보고 알려 줘.
        - 한국 원자력 연구원에서 선량 평가 시스템 고도화 사업을 발주했는데, 이 사업이 왜 추진되는지 목적을 알려 줘.
        - 고려대학교 차세대 포털 시스템 사업이랑 광주과학기술원의 학사 시스템 기능개선 사업을 비교해 줄래?
            - 고려대학교랑 광주과학기술원 각각 응답 시간에 대한 요구사항이 있나? 문서를 기반으로 정확하게 답변해 줘.
- 품질 좋은 답변도 중요하지만 응답 속도가 너무 느려져서도 안 됩니다.

In [1]:
# Google Drive Mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import os

data_dir = '/content/drive/MyDrive/Data' # Updated path
print(f"Listing contents of directory: {data_dir}")
try:
    for item in os.listdir(data_dir):
        print(item)
except FileNotFoundError:
    print(f"Directory not found: {data_dir}")
except Exception as e:
    print(f"An error occurred while listing directory contents: {e}")

Listing contents of directory: /content/drive/MyDrive/Data
files
data_list.csv
data_list.gsheet


# Install Packages

In [3]:
!pip install langchain langchain-openai easyocr pyhwp hwp-extract pymupdf pymupdf4llm faiss-cpu langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.1/218.1 kB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 62.0 MB/s eta 0:00:00
   ━━━

In [4]:
# 필수 빌드 도구 & 라이브러리 설치
!apt-get update
!apt-get install -y build-essential libgsf-1-dev libxml2-dev libglib2.0-dev libiconv-hook-dev

# 소스코드 clone
!git clone https://github.com/mete0r/pyhwp.git


# 설치
!python setup.py install

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,006 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ub

## Load data_list.csv and find NaN values

In [5]:
import pandas as pd
import os

data_dir_path = '/content/drive/MyDrive/Data' # Updated path
csv_path = os.path.join(data_dir_path, 'data_list.csv')

try:
    # Load the CSV file into a pandas DataFrame
    data_list_df = pd.read_csv(csv_path)

    # Display the first few rows of the DataFrame
    print("First 5 rows of data_list.csv:")
    display(data_list_df.head())

    # Check for NaN values in each column
    print("\nChecking for NaN values:")
    nan_counts = data_list_df.isnull().sum()

    # Display the count of NaN values per column
    print("Number of NaN values per column:")
    print(nan_counts)

except FileNotFoundError:
    print(f"Error: The file {csv_path} was not found.")
except Exception as e:
    print(f"An error occurred while reading the CSV file or checking for NaN values: {e}")

First 5 rows of data_list.csv:


,공고 번호,공고 차수,사업명,사업 금액,발주 기관,공개 일자,입찰 참여 시작일,입찰 참여 마감일,사업 요약,파일형식,파일명,텍스트
0,20241001798,0.0,한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보시스템 고도화,130000000.0,한영대학,2024-10-04 13:51:23,NaN,2024-10-15 17:00:00,- 한영대학교 특성화 맞춤형 교육환경 구축을 위해 트랙운영 학사정보시스템을 고도화한...,hwp,한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp,\n \n2024년 특성화 맞춤형 교육환경 구축 – 트랙운영 학사정보시스템 ...
1,20241002912,0.0,2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선,129300000.0,한국연구재단,2024-10-04 15:01:52,2024-10-14 10:00:00,2024-10-16 14:00:00,- 사업 개요: 2024년 대학 산학협력활동 실태조사 시스템(UICC) 기능개선\n...,hwp,한국연구재단_2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선.hwp,\r\n \r\n \r\n \r\n제 안 요 청 서\r\n[ 2024년 대학 ...
2,20240827859,0.0,EIP3.0 고압가스 안전관리 시스템 구축 용역,40000000.0,한국생산기술연구원,2024-08-28 11:31:02,2024-08-29 09:00:00,2024-09-09 10:00:00,- 사업 개요: EIP3.0 고압가스 안전관리 시스템 구축 용역\n- 추진배경: 안...,hwp,한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.hwp,\r\n \r\nEIP3.0 고압가스 안전관리\r\n시스템 구축 용역\...
3,20240430918,0.0,도시계획위원회 통합관리시스템 구축용역,150000000.0,인천광역시,2024-04-18 16:26:32,2024-05-02 10:00:00,2024-05-09 16:00:00,- 사업명: 도시계획위원회 통합관리시스템 구축 용역\n- 용역개요: 도시계획위원회와...,hwp,인천광역시_도시계획위원회 통합관리시스템 구축용역.hwp,\r\n \r\n \r\n도시계획위원회 통합관리시스템 구축\r\n제 안 요 청...
4,20240430896,0.0,봉화군 재난통합관리시스템 고도화 사업(협상)(긴급),900000000.0,경상북도 봉화군,2024-04-18 16:33:28,2024-04-26 09:00:00,2024-04-30 17:00:00,- 사업명: 봉화군 재난통합관리시스템 고도화 사업\n- 사업개요: 공동수급(공동이행...,hwp,경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).hwp,\r\n \r\n \r\n제안요청서\r\n \r\n사 업 명\r\n봉화...



Checking for NaN values:
Number of NaN values per column:
공고 번호        18
공고 차수        18
사업명           0
사업 금액         1
발주 기관         0
공개 일자         0
입찰 참여 시작일    26
입찰 참여 마감일     8
사업 요약         0
파일형식          0
파일명           0
텍스트           0
dtype: int64


# Load and Process Documents (using Langchain)

In [50]:
import os
import sys
import fitz  # PyMuPDF
import easyocr
import numpy as np
from PIL import Image
import subprocess
from langchain.schema import Document
# Reverting to pyhwp for HWP extraction
import pyhwp
import pandas as pd # Import pandas for data_list_df usage
import re # Import re for text cleaning


def find_executable(executable_name):
    """Finds the full path to an executable within the environment's PATH or script directories."""
    # Check in PATH first
    for path in os.environ["PATH"].split(os.pathsep):
        exe_path = os.path.join(path, executable_name)
        if os.path.isfile(exe_path) and os.access(exe_path, os.X_OK):
            return exe_path

    # Check in common script directories relative to sys.executable
    script_dirs = [
        os.path.join(os.path.dirname(sys.executable), 'bin'), # Linux/macOS
        os.path.join(os.path.dirname(sys.executable), 'Scripts') # Windows
    ]
    for script_dir in script_dirs:
        exe_path = os.path.join(script_dir, executable_name)
        if os.path.isfile(exe_path) and os.access(exe_path, os.X_OK):
            return exe_path

    return None

def clean_text(text):
    """
    Cleans extracted text by removing excessive whitespace and potentially noise characters,
    including single quotes and dots within text, and specific long dot sequences.
    """
    # Remove excessive whitespace (spaces, tabs, newlines)
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove potential noise characters (e.g., single dots that are not part of ellipses or punctuation)
    # This is a basic approach; more sophisticated methods might be needed depending on the noise.
    text = re.sub(r'(?<!\.)\.(?!\.)', '', text) # Remove single dots not part of an ellipsis - This might be too aggressive
    text = text.replace('·', '') # Remove specific dot character often seen in HWP conversions

    # Remove single quotes
    text = text.replace("'", "")

    # Remove dots that are not preceded or followed by a space or are not part of a number (heuristic)
    # This is a more aggressive removal and might remove some valid dots in abbreviations etc.
    # Consider refining this regex based on specific noise patterns observed.
    text = re.sub(r'(?<!\s)\.(?!\s)', '', text) # Removing this as it might be too aggressive
    text = re.sub(r'(?<!\d)\.(?!\d)', '', text) # Removing this as it might be too aggressive

    # Remove occurrences of "<표>" and "<그림>" tags
    text = text.replace("<표>", "").replace("<그림>", "")


    # Add more specific cleaning for long sequences of dots or similar noise
    # This regex targets sequences of 2 or more non-alphanumeric characters (including dots, dashes, etc.)
    # You might need to adjust this regex based on the specific noise patterns you observe
    text = re.sub(r'[^a-zA-Z0-9\s]{2,}', '', text)

    # More aggressive removal of multiple dots or similar characters
    text = re.sub(r'\.{2,}', '', text) # Remove sequences of 2 or more dots
    text = re.sub(r'[-]{2,}', '', text) # Remove sequences of 2 or more dashes
    text = re.sub(r'[·]{2,}', '', text) # Remove sequences of 2 or more middle dots


    return text


# Define HWP extraction function using pyhwp (basic text extraction)
def extract_text_from_hwp(hwp_path):
    """
    Extracts text from an HWP file using pyhwp's basic text extraction.
    Tries hwp5txt command-line tool first as it's generally more reliable.

    Args:
        hwp_path (str): The path to the HWP file.

    Returns:
        str: The extracted text from the HWP.
    """
    text = ""
    hwp5txt_path = find_executable('hwp5txt')

    if hwp5txt_path:
        try:
            # Use hwp5txt command-line tool via subprocess
            result = subprocess.run([hwp5txt_path, hwp_path], capture_output=True, text=True, encoding='utf-8')
            if result.returncode == 0:
                text = result.stdout
            else:
                print(f"Warning: hwp5txt failed for {hwp_path}: {result.stderr}")
                text = "" # Fallback to empty if hwp5txt fails
        except Exception as e:
            print(f"Warning: Error running hwp5txt for {hwp_path}: {e}")
            text = "" # Fallback to empty if hwp5txt throws exception
    else:
        print("Warning: hwp5txt command not found. HWP extraction might not work as expected.")
        text = "" # Fallback to empty if hwp5txt is not found

    return clean_text(text) # Apply cleaning


# Define function for PDF text extraction using easyOCR
def extract_text_from_pdf_easyocr(pdf_path):
    """
    Extracts text from a PDF file using easyOCR.

    Args:
        pdf_path (str): The path to the PDF file.

    Returns:
        str: The extracted text from the PDF.
    """
    # Initialize easyocr reader for Korean and English (adjust languages as needed)
    reader = easyocr.Reader(['ko', 'en'])
    text = ""
    try:
        pdf_document = fitz.open(pdf_path)
        for page_num in range(pdf_document.page_count):
            page = pdf_document.load_page(page_num)

            # Render page to an image (as a numpy array) for easyOCR
            pixmap = page.get_pixmap()
            img = Image.frombytes("RGB", [pixmap.width, pixmap.height], pixmap.samples)
            img_np = np.array(img)

            # Use easyOCR to read text from the image
            result = reader.readtext(img_np, detail=0) # detail=0 returns only the text
            text += " ".join(result) + "\n" # Join lines with a space and add newline between pages

        pdf_document.close()
    except Exception as e:
        print(f"Error processing PDF {pdf_path} with easyOCR: {e}")
        text = "" # Return empty string in case of processing error

    return clean_text(text) # Apply cleaning


data_dir = '/content/drive/MyDrive/Data/files' # Updated path
csv_path = '/content/drive/MyDrive/Data/data_list.csv' # Path to data_list.csv

try:
    # Load data_list.csv into a pandas DataFrame
    data_list_df = pd.read_csv(csv_path)
except FileNotFoundError:
    print(f"Error: data_list.csv not found at {csv_path}")
    data_list_df = pd.DataFrame() # Create an empty DataFrame if file not found
except Exception as e:
    print(f"An error occurred while loading data_list.csv: {e}")
    data_list_df = pd.DataFrame() # Create an empty DataFrame on error


processed_documents = []

# Specify the files to process - based on previous examples and user request for top 4 HWP and top 1 PDF
# This assumes the order in data_list_df reflects the "top" files the user wants to process.
if not data_list_df.empty:
    hwp_files_to_process = data_list_df[data_list_df['파일형식'] == 'hwp']['파일명'].tolist()[:4]
    pdf_files_to_process = data_list_df[data_list_df['파일형식'] == 'pdf']['파일명'].tolist()[:1]
    files_to_process = hwp_files_to_process + pdf_files_to_process
else:
    files_to_process = []
    print("Cannot determine files to process as data_list.csv was not loaded.")


print(f"Attempting to process specified files from directory: {data_dir}")
print(f"Files selected for processing: {files_to_process}")

if not os.path.isdir(data_dir):
    print(f"Error: Directory not found at {data_dir}")
else:
    for filename in files_to_process:
        file_path = os.path.join(data_dir, filename)
        if os.path.isfile(file_path):
            # Get metadata for the current file
            file_metadata = data_list_df[data_list_df['파일명'] == filename].iloc[0].to_dict() if filename in data_list_df['파일명'].values else {}

            if filename.lower().endswith('.pdf'):
                try:
                    text = extract_text_from_pdf_easyocr(file_path) # Use easyOCR function
                    if text:
                         # Add metadata to the Document object
                        processed_documents.append(Document(page_content=text, metadata={"source": file_path, **file_metadata}))
                        print(f"Processed PDF: {filename}")
                except Exception as e:
                    print(f"Failed to process PDF {filename} with easyOCR: {e}")
            elif filename.lower().endswith('.hwp'):
                try:
                    text = extract_text_from_hwp(file_path) # Use pyhwp function (via hwp5txt)
                    if text:
                         # Add metadata to the Document object
                        processed_documents.append(Document(page_content=text, metadata={"source": file_path, **file_metadata}))
                        print(f"Processed HWP: {filename}")
                except Exception as e:
                    print(f"Failed to process HWP {filename} with pyhwp (hwp5txt): {e}")
            else:
                print(f"Skipping unsupported file type: {filename}")
        else:
            print(f"Warning: Specified file not found: {filename}")


print("\nDocument processing complete.")
print(f"Total documents processed: {len(processed_documents)}")

# Display one HWP and one PDF example with metadata if available
print("\nExamples of processed documents with metadata:")
hwp_example = next((doc for doc in processed_documents if doc.metadata.get('파일형식') == 'hwp'), None)
pdf_example = next((doc for doc in processed_documents if doc.metadata.get('파일형식') == 'pdf'), None)

if hwp_example:
    print("--- HWP Document Example ---")
    print(f"Content snippet: {hwp_example.page_content[:500]}...")
    print(f"Metadata: {hwp_example.metadata}")
else:
    print("No HWP documents were successfully processed with metadata.")

if pdf_example:
    print("\n--- PDF Document Example ---")
    print(f"Content snippet: {pdf_example.page_content[:500]}...")
    print(f"Metadata: {pdf_example.metadata}")
else:
    print("\nNo PDF documents were successfully processed with metadata.")

Attempting to process specified files from directory: /content/drive/MyDrive/Data/files
Files selected for processing: ['한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp', '한국연구재단_2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선.hwp', '한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.hwp', '인천광역시_도시계획위원회 통합관리시스템 구축용역.hwp', '고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf']
Processed HWP: 한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp
Processed HWP: 한국연구재단_2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선.hwp
Processed HWP: 한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.hwp
Processed HWP: 인천광역시_도시계획위원회 통합관리시스템 구축용역.hwp
Processed PDF: 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf

Document processing complete.
Total documents processed: 5

Examples of processed documents with metadata:
--- HWP Document Example ---
Content snippet:  2024 10      □ 사 업 명 :      -    □  : 130,000,000원  내 (VAT  □  :  3  1  *  및        수  □  :                  및      및          및        및      ◦    및     ◦          ◦            ◦    및         ◦          /  /  /   ◦      및     함 ◦       및     ❍  :  3 

In [51]:
# Re-run Embedding Generation and Vector DB (FAISS) Creation
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from google.colab import userdata

try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except userdata.SecretNotFoundError:
    print("Error: OPENAI_API_KEY not found in Colab secrets.")
    print("Please add your OpenAI API key to the Colab secrets manager (🔑 icon on the left) with the name 'OPENAI_API_KEY'.")
    OPENAI_API_KEY = None


if OPENAI_API_KEY and semantic_chunked_documents:
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)
    vectorstore = FAISS.from_documents(semantic_chunked_documents, embeddings)

    print("FAISS vector store created with embeddings from semantic chunks.")
    print(f"Number of documents in vector store: {vectorstore.index.ntotal}")

else:
    print("Skipping vector store creation due to missing API key or no semantic chunks.")
    vectorstore = None

FAISS vector store created with embeddings from semantic chunks.
Number of documents in vector store: 303


In [52]:
# Re-run Advanced Retrieval (using Langchain)
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers import MultiQueryRetriever
from langchain.prompts import PromptTemplate
from google.colab import userdata

try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except userdata.SecretNotFoundError:
    print("Error: OPENAI_API_KEY not found in Colab secrets.")
    print("Please add your OpenAI API key to the Colab secrets manager (🔑 icon on the left) with the name 'OPENAI_API_KEY'.")
    OPENAI_API_KEY = None

if OPENAI_API_KEY and vectorstore:
    llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY, temperature=0.3, max_tokens=1500, top_p=0.8)

    template = """다음 문서를 사용하여 질문에 답하세요.
    답변은 가능한 한 정확하고 간결하게 작성하세요.
    제공된 문서에 없는 내용은 언급하지 마세요.
    문서:
    {context}

    질문: {question}

    답변:"""
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

    base_retriever = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 10, "fetch_k": 20}
    )

    multiquery_retriever = MultiQueryRetriever.from_llm(
        retriever=base_retriever,
        llm=llm
    )

    final_retriever = multiquery_retriever

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=final_retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
    )

    print("Advanced RetrievalQA chain created with Multi-Query, MMR, and refined generation parameters/prompt.")

else:
    print("Skipping Advanced RAG chain creation due to missing API key, vector store, or no semantic chunks.")
    qa_chain = None

Advanced RetrievalQA chain created with Multi-Query, MMR, and refined generation parameters/prompt.


In [58]:
import time

def time_rag_query(qa_chain, query):
    """
    Times the execution of a RAG query.

    Args:
        qa_chain: The RAG chain object.
        query (str): The query string.

    Returns:
        tuple: A tuple containing the response dictionary and the execution time in seconds.
    """
    start_time = time.time()
    response = qa_chain.invoke({"query": query})
    end_time = time.time()
    execution_time = end_time - start_time
    return response, execution_time

In [53]:
# Re-run Example Query 1
if qa_chain and processed_documents:
    hwp1_doc = next((doc for doc in processed_documents if '한영대학' in doc.metadata.get('파일명', '')), None)
    if hwp1_doc:
        query = "한영대학교 트랙운영 학사정보시스템 고도화 사업의 추진 배경 및 필요성은 무엇인가요?"
        response, query_time = time_rag_query(qa_chain, query)

        print("Query:")
        print(query)
        print("\nGenerated Response:")
        print(response['result'])
        print(f"\nQuery Time: {query_time:.2f} seconds")
        print("\nSource Documents:")
        for doc in response['source_documents']:
            print(f"--- Source: {doc.metadata.get('source', 'N/A')} (Chunk Index: {doc.metadata.get('chunk_index', 'N/A')}) ---")
            print(doc.page_content[:300] + "...")
    else:
        print("Could not find processed document for 한영대학 to generate query.")
else:
    print("RAG chain or processed documents not initialized.")

Query:
한영대학교 트랙운영 학사정보시스템 고도화 사업의 추진 배경 및 필요성은 무엇인가요?

Generated Response:
한영대학교 트랙운영 학사정보시스템 고도화 사업의 추진 배경 및 필요성은 다음과 같습니다:

1. **교육 환경의 급격한 변화**: 학생 수 감소, 코로나19 팬데믹으로 인한 교육 환경 변화, 인공지능의 활용 등으로 대학이 급격한 사회-기술 변화에 직면하고 있습니다.

2. **분산된 시스템 및 데이터의 통합 필요**: 정보화 요구가 증가하고 있으나, 현재 노후화된 학사 시스템으로 인해 업무가 분산되어 있어 시스템 통합에 대한 필요성이 커지고 있습니다.

3. **데이터 기반 대학 경영 지원 개선**: 분절된 정보 시스템 간의 정보 연계 미비 및 정보화 표준 부재로 인해 경영 현황 파악 및 의사결정 시 한계가 발생하고 있습니다.

4. **사용자 정보 서비스 접근성 개선**: 교내 구성원의 정보 접근 지원을 위한 포털 서비스가 제공되고 있으나, 정보 접근성 및 유용성에 대한 개선 요구가 상당합니다.

5. **대학 경쟁력 강화 및 전략 목표 달성 지원**: 노후화된 시스템의 차세대 구축을 통해 정보 서비스의 품질을 강화하고 대학 교육 시스템의 경쟁력을 확보하고자 합니다.

Query Time: 9.74 seconds

Source Documents:
--- Source: /content/drive/MyDrive/Data/files/한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp (Chunk Index: N/A) ---
2024 10      □ 사 업 명 : 한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보시스템 고도화 □ 사업예산 : 130,000,000원 범위 내 (VAT 포함) □ 사업기간 : 계약일로부터 3개월 (안정화기간 1개월 포함) * 기간 및 일정은 학교 사정과 용역대상자와의 협의에 따라 조정될 수 있음 □ 입찰방법 : 제한경쟁입찰(협상에 의한 계약 체결)  학사제도제도개편과 

In [54]:
# Re-run Example Query 2
if qa_chain and processed_documents:
    hwp2_doc = next((doc for doc in processed_documents if '한국연구재단' in doc.metadata.get('파일명', '')), None)
    if hwp2_doc:
        query = "한국연구재단의 대학산학협력활동 실태조사 시스템 기능개선 사업의 목표는 무엇인가요?"
        response, query_time = time_rag_query(qa_chain, query)

        print("Query:")
        print(query)
        print("\nGenerated Response:")
        print(response['result'])
        print(f"\nQuery Time: {query_time:.2f} seconds")
        print("\nSource Documents:")
        for doc in response['source_documents']:
            print(f"--- Source: {doc.metadata.get('source', 'N/A')} (Chunk Index: {doc.metadata.get('chunk_index', 'N/A')}) ---")
            print(doc.page_content[:300] + "...")
    else:
        print("Could not find processed document for 한국연구재단 to generate query.")
else:
    print("RAG chain or processed documents not initialized.")

Query:
한국연구재단의 대학산학협력활동 실태조사 시스템 기능개선 사업의 목표는 무엇인가요?

Generated Response:
한국연구재단의 대학산학협력활동 실태조사 시스템 기능개선 사업의 목표는 UICC 시스템 기능개선을 통해 이용 편의성을 개선하는 것입니다.

Query Time: 5.81 seconds

Source Documents:
--- Source: /content/drive/MyDrive/Data/files/한국연구재단_2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선.hwp (Chunk Index: N/A) ---
2024 10       1 사업명: 2024년 대학 산학협력활동 실태조사 시스템(UICC) 기능개선 2 추진배경 및 필요성 □ 한국연구재단법 제5조, 산업교육진흥 및 산학연협력촉진에 관한 법률 제39조의2 및 제43조에 따라 실시하는 대학 산학협력활동 실태조사를 안정적으로 운영 필요 □ 항목 지침 변경사항 등 변화에 신속하고 정확하게 대응하며, 기능개선에 대한 사용자 요구를 반영하여 사용자 편의성 강화 필요 □ UICC의 효율적인 관리와 운영을 위한 지원 체계 및 각종 기관 요구사항에 대응 필요 3 주관기관 : 한국연구재단 4 사...
--- Source: /content/drive/MyDrive/Data/files/고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf (Chunk Index: N/A) ---
반찬계좌 보이거 탑 교녀고외장학생관리어서 결의서작성 연결 처리루 전자걸재 기능 필요 수업표 지급방식 잠학금은 대출계좌 채번R점 승인 구 전자결재 진행 다량의 환수 더이터롬 일관 전표처리관 수 있거 설계 필요 중복지원 심사(지급) 정보 파일 다문로드 대출처번계좌 정보파일 업로드 등록 반환 결의시 잠학금이 있는 경무 장학금 순위에 따라 잠학금계 좌로 이제하는 내용 설계 필요 (SFR-학사-032 등록금수님관리) 등록: 등록금 반퀴결의 잠학: 장학금 지출결의 별도로 하지만 티부서 담당자들에서 통합 메뉴로 보이도록

In [55]:
# Re-run Example Query 3
if qa_chain and processed_documents:
    hwp3_doc = next((doc for doc in processed_documents if '한국생산기술연구원' in doc.metadata.get('파일명', '')), None)
    if hwp3_doc:
        query = "한국생산기술연구원의 EIP3.0 고압가스 안전관리 시스템 구축 용역의 사업 개요를 설명해 주세요."
        response, query_time = time_rag_query(qa_chain, query)

        print("Query:")
        print(query)
        print("\nGenerated Response:")
        print(response['result'])
        print(f"\nQuery Time: {query_time:.2f} seconds")
        print("\nSource Documents:")
        for doc in response['source_documents']:
            print(f"--- Source: {doc.metadata.get('source', 'N/A')} (Chunk Index: {doc.metadata.get('chunk_index', 'N/A')}) ---")
            print(doc.page_content[:300] + "...")
    else:
        print("Could not find processed document for 한국생산기술연구원 to generate query.")
else:
    print("RAG chain or processed documents not initialized.")

Query:
한국생산기술연구원의 EIP3.0 고압가스 안전관리 시스템 구축 용역의 사업 개요를 설명해 주세요.

Generated Response:
한국생산기술연구원의 EIP3.0 고압가스 안전관리 시스템 구축 용역은 고압가스 취급 부주의로 인한 연구실 사고를 예방하고, 법적 요구사항을 준수하기 위해 체계적인 관리 시스템을 구축하는 사업입니다. 이 사업의 주요 목표는 다음과 같습니다:

1. 원내에서 관리하는 화학물질 운영현황의 통계 정보를 제공하는 시스템 구축.
2. 고압가스 화학물질의 정보와 판매허가 업체를 관리하는 시스템 구축.
3. 고압가스 화학물질의 구매부터 안전성 검토 과정을 진행하기 위한 시스템 고도화.
4. 연구실과 법적 준수 사항을 관리하기 위한 시스템 고도화.
5. 화학제품 폐기·회수를 관리하기 위한 시스템 고도화.

이 사업을 통해 연구원들의 생명과 신체를 보호하고, 연구실 내 안전사고를 예방하며, 법적 문제를 예방하고 안전 관리 수준을 법적 기준 이상으로 유지하는 것을 목표로 하고 있습니다.

Query Time: 8.74 seconds

Source Documents:
--- Source: /content/drive/MyDrive/Data/files/한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.hwp (Chunk Index: N/A) ---
시스템 <업무 메뉴 및 전자결재 운영 현황 (20221231기준)>  * 연구관리, 기업지원, 재무, 인사, 급여, 구매조달, 지재권 등 19개 업무시스템 운영 中 ○ 연계시스템 (20231231 기준)  ○ 장비 현황 - 운용 중인 전산장비는 총 1,205종이며, 840종을 서비스수준협약(SLA)을 통하여 안정적인 서비스와 신속한 장애 복구 체계 구축  3 사업 추진방안 가 추진목표 ㅇ 원내에서 관리하는 화학물질 운영현황의 통계 정보 공유 ㅇ 고압가스 화학물질의 정보와 판매허가 업체를 관리 ㅇ 고압가스 화학물질의 구매부터 안전성 ...
--- Source: /content/dr

In [56]:
# Re-run Example Query 4
if qa_chain and processed_documents:
    hwp4_doc = next((doc for doc in processed_documents if '인천광역시' in doc.metadata.get('파일명', '')), None)
    if hwp4_doc:
        query = "인천광역시 도시계획위원회 통합관리시스템 구축 용역의 용역 개요는 무엇인가요?"
        response, query_time = time_rag_query(qa_chain, query)

        print("Query:")
        print(query)
        print("\nGenerated Response:")
        print(response['result'])
        print(f"\nQuery Time: {query_time:.2f} seconds")
        print("\nSource Documents:")
        for doc in response['source_documents']:
            print(f"--- Source: {doc.metadata.get('source', 'N/A')} (Chunk Index: {doc.metadata.get('chunk_index', 'N/A')}) ---")
            print(doc.page_content[:300] + "...")
    else:
        print("Could not find processed document for 인천광역시 to generate query.")
else:
    print("RAG chain or processed documents not initialized.")

Query:
인천광역시 도시계획위원회 통합관리시스템 구축 용역의 용역 개요는 무엇인가요?

Generated Response:
인천광역시 도시계획위원회 통합관리시스템 구축 용역의 용역 개요는 다음과 같습니다:
- 용역명: 도시계획위원회 통합관리시스템 구축 용역
- 기간: 착수일로부터 180일
- 사업비: 금150,000,000원 (VAT 포함)
- 계약방법: 제한경쟁입찰(협상에 의한 계약)

Query Time: 6.63 seconds

Source Documents:
--- Source: /content/drive/MyDrive/Data/files/인천광역시_도시계획위원회 통합관리시스템 구축용역.hwp (Chunk Index: N/A) ---
1 과 업 명: 「도시계획위원회 통합관리시스템 구축 용역」 2 용역개요 ○ 용 역 명: 도시계획위원회 통합관리시스템 구축 용역 ○ 기 간: 착수일로부터 180일 ○ 사 업 비: 금150,000,000원 (VAT 포함) ○ 계약방법: 제한경쟁입찰(협상에 의한 계약)    3 과업배경 및 목적 ○ (수작업에 의한 업무의 복잡성) 여러 단계의 위원회 운영업무를 수작업으로 처리, 업무의 복잡성 발생 ‧ 심의자료 변경 시 자료의 재배포 등 업무의 중복성 발생 ☞ 위원회 전 과정의 지원체계 구축 필요 ○ (통합관리시스템 부재로 업무별 개별 처...
--- Source: /content/drive/MyDrive/Data/files/고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf (Chunk Index: N/A) ---
연계복록 및 함부의 추가 변경 삭제 절차의 정의 절차분 수립하여이 합 연계 시스템 도입 시 시스템과 연제터 체계 수립 오류 데이터의 진단 및 기신 절차늘 저시하여야 탑 데이터 구조관리 방안(매뉴벌 가이드 틈) 데이터 표준 구조 면계 , 문질) 산출정보 관리 제계서 요구사항 분류 데이터 R구사람 요구사항 번호 DAF-UD5 요구사항 명칭 연계 데이터 관리체계 정의 연계 데이터 관리체계 수립 데이터 연제 관리 방만

In [57]:
# Re-run Example Query 5
if qa_chain and processed_documents:
    pdf_doc = next((doc for doc in processed_documents if '고려대학교' in doc.metadata.get('파일명', '')), None)
    if pdf_doc:
        query = "고려대학교 차세대 포털·학사 정보시스템 구축 사업의 추진 배경을 설명해주세요."
        response, query_time = time_rag_query(qa_chain, query)

        print("Query:")
        print(query)
        print("\nGenerated Response:")
        print(response['result'])
        print(f"\nQuery Time: {query_time:.2f} seconds")
        print("\nSource Documents:")
        for doc in response['source_documents']:
            print(f"--- Source: {doc.metadata.get('source', 'N/A')} (Chunk Index: {doc.metadata.get('chunk_index', 'N/A')}) ---")
            print(doc.page_content[:300] + "...")
    else:
        print("Could not find processed document for 고려대학교 to generate query.")
else:
    print("RAG chain or processed documents not initialized.")

Query:
고려대학교 차세대 포털·학사 정보시스템 구축 사업의 추진 배경을 설명해주세요.

Generated Response:
고려대학교 차세대 포털·학사 정보시스템 구축 사업의 추진 배경은 다음과 같습니다:

1. **교육 환경의 급격한 변화**: 인구 감소, 코로나19 팬데믹으로 인한 교육 환경 변화, 인공지능의 활용 등으로 대학이 급격한 사회-기술 변화에 직면하고 있습니다.

2. **분산된 시스템 및 데이터의 통합 필요성**: 정보화 요구가 증가하고 있지만, 기존의 노후화된 학사 시스템에 기반하여 업무가 분산되어 있어 시스템 통합에 대한 요구가 커지고 있습니다.

3. **데이터 기반 대학 경영 지원 개선**: 분절된 정보 시스템 간의 정보 연계 미흡 및 정보화 표준 부재로 인해 경영 현황 파악 및 의사결정 시 한계가 발생하고 있습니다.

4. **사용자 정보 서비스 접근성 개선**: 교내 구성원의 정보 탐색 지원을 위한 포털 서비스가 제공되고 있으나, 정보 접근성과 유용성에 대한 개선 요구가 상당합니다.

5. **대학 경쟁력 강화 및 전략 목표 달성 지원**: 노후화된 시스템의 차세대 구축을 통해 정보 서비스의 품질을 강화하고, 대학의 교육 시스템 경쟁력을 확보하고자 합니다.

Query Time: 21.61 seconds

Source Documents:
--- Source: /content/drive/MyDrive/Data/files/고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf (Chunk Index: N/A) ---
고려대학교 ROa UIVER SITY 제 안 요 청 서 고려대학교 차세대 포털 학사 정보시스템 구축 사업 2024 7 07 aE 고려대학교 KOREA UNIVERSITY 본 자고; 게민내공의 신편움 취한 "포자고로 부거으로 '년부지 사용타; 형위: 구립- 고려대학교 ROa UIVER SITY 목 차 사업개요 1 사업 개요 2 사업 배경 3 사업 범위 : 기대 증과 I 현황 및 문제점 1 일반 된참  2 점수화 된참 문제

# Implement Semantic Chunking

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# While true semantic chunking often requires deeper document structure analysis
# or advanced models, we can use RecursiveCharacterTextSplitter with parameters
# tuned to try and respect some structural elements like paragraphs or headings
# by splitting on various characters including newlines.

# You could potentially add logic here to pre-process text based on HTML structure
# if using hwp5html, or analyze PDF layout for more semantic splits.
# For this step, we'll use RecursiveCharacterTextSplitter with more aggressive
# splitting characters to try and capture semantic breaks.

# Initialize a text splitter with additional separators that might indicate semantic breaks
# The order of separators matters: try splitting on larger semantic units first.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Adjust chunk size as needed
    chunk_overlap=100, # Adjust overlap as needed
    separators=["\n\n", "\n", " ", ""] # Try splitting on paragraphs, then lines, then spaces
)

# Split the processed documents into chunks
# Note: If using hwp5html, the page_content will be HTML, and this splitter
# might not optimally handle HTML tags as semantic separators.
# A dedicated HTML splitter or pre-processing might be needed for better results with HTML.
semantic_chunked_documents = text_splitter.split_documents(processed_documents)

print(f"Original documents: {len(processed_documents)}")
print(f"Semantic chunked documents: {len(semantic_chunked_documents)}")

# Display first few semantic chunked documents
if semantic_chunked_documents:
    print("\nFirst 3 semantic chunked documents example:")
    for i in range(min(3, len(semantic_chunked_documents))):
        print(f"--- Semantic Chunk {i+1} ---")
        print(f"Content snippet: {semantic_chunked_documents[i].page_content[:500]}...")
        print(f"Metadata: {semantic_chunked_documents[i].metadata}")

Original documents: 5
Semantic chunked documents: 391

First 3 semantic chunked documents example:
--- Semantic Chunk 1 ---
Content snippet: <표>





          


2024. 10.




<그림>

 

<표>






<그림>


<표>



<표>


□ 사 업 명 : 한영대학교 특성화 맞춤형 교육환경 구축 - 
             트랙운영 학사정보시스템 고도화
□ 사업예산 : 130,000,000원 범위 내 (VAT 포함)
□ 사업기간 : 계약일로부터 3개월 (안정화기간 1개월 포함)
  * 기간 및 일정은 학교 사정과 용역대상자와의 협의에 따라 조정될 수 있음
□ 입찰방법 : 제한경쟁입찰(협상에 의한 계약 체결)



<표>


학사제도･제도개편과 연계하여 전공교과목 선택폭을 넓히고, 트랙제 교육과정 참여자에게 다양한 진로선택의 기회를 제공 및 취업문 확대 
트랙제 교육과정의 도입 및 운영으로 산업현장의 경쟁력 강화
산업체 수요 맞춤 교육과정 운영 및 활성화로 교육과정 내실화 
기업수요 연계 확대로 산업체 및 지역사회 현장실무형 인재 양성 



<표>


 ◦ 트랙기반 교육과정의 운영 및 ...
Metadata: {'source': '/content/drive/MyDrive/Data/files/한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp', '공고 번호': '20241001798', '공고 차수': 0.0, '사업명': '한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보시스템 고도화', '사업 금액': 130000000.0, '발주 기관': '한영대학', '공개 일자': '2024-10-04 13:51:23', '입찰 참여 시작일': nan, '입찰 참여 마감일': '2024-10-15 17:00:00', '사업 요약': '- 한영대학교 특성화 맞춤형 교육환경 구축을 위해 트랙

# Embedding Generation and Vector DB (FAISS) Creation (using Langchain)

In [11]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from google.colab import userdata

# Access your API key from Colab secrets
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except userdata.SecretNotFoundError:
    print("Error: OPENAI_API_KEY not found in Colab secrets.")
    print("Please add your OpenAI API key to the Colab secrets manager (🔑 icon on the left) with the name 'OPENAI_API_KEY'.")
    OPENAI_API_KEY = None # Set to None to prevent further errors


if OPENAI_API_KEY:
    # Initialize the OpenAI embeddings model
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)

    # Create a FAISS vector store from the semantically chunked documents
    # This will generate embeddings and add them to the FAISS index
    vectorstore = FAISS.from_documents(semantic_chunked_documents, embeddings)

    print("FAISS vector store created with embeddings from semantic chunks.")
    print(f"Number of documents in vector store: {vectorstore.index.ntotal}")

    # You can optionally save the vector store to disk for later use
    # vectorstore.save_local("semantic_faiss_index")

else:
    print("Skipping vector store creation due to missing API key.")
    vectorstore = None # Set vectorstore to None if API key is missing

FAISS vector store created with embeddings from semantic chunks.
Number of documents in vector store: 391


# Implement Advanced Retrieval (using Langchain)

In [12]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers import MultiQueryRetriever
from langchain.prompts import PromptTemplate # Import PromptTemplate
from google.colab import userdata

# Access your API key from Colab secrets
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except userdata.SecretNotFoundError:
    print("Error: OPENAI_API_KEY not found in Colab secrets.")
    print("Please add your OpenAI API key to the Colab secrets manager (🔑 icon on the left) with the name 'OPENAI_API_KEY'.")
    OPENAI_API_KEY = None # Set to None to prevent further errors

if OPENAI_API_KEY and vectorstore:
    # Initialize the ChatOpenAI model for generation (will also be used for Multi-Query)
    # Adjusting parameters for generation enhancement
    llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY, temperature=0.3, max_tokens=1500, top_p=0.8) # Adjusted temp, max_tokens, added top_p

    # Define a refined prompt template
    # This template guides the language model on how to use the retrieved context
    template = """다음 문서를 사용하여 질문에 답하세요.
    답변은 가능한 한 정확하고 간결하게 작성하세요.
    제공된 문서에 없는 내용은 언급하지 마세요.
    문서:
    {context}

    질문: {question}

    답변:"""
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


    # Base Retriever with higher k and MMR
    base_retriever = vectorstore.as_retriever(
        search_type="mmr", # Use MMR search
        search_kwargs={"k": 10, "fetch_k": 20} # Increased k and fetch_k
    )

    # Implement Multi-Query Retriever
    multiquery_retriever = MultiQueryRetriever.from_llm(
        retriever=base_retriever,
        llm=llm
    )

    # Using the multi-query retriever as the main retriever for now
    # Metadata filtering would be integrated here if implemented dynamically
    final_retriever = multiquery_retriever # Use the multi-query retriever

    # Create a RetrievalQA chain with the advanced retriever and refined prompt
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=final_retriever, # Use the advanced retriever
        return_source_documents=True,
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT} # Pass the refined prompt
    )

    print("Advanced RetrievalQA chain created with Multi-Query, MMR, and refined generation parameters/prompt.")

else:
    print("Skipping Advanced RAG chain creation due to missing API key or vector store.")
    qa_chain = None

Advanced RetrievalQA chain created with Multi-Query, MMR, and refined generation parameters/prompt.


# Example Queries Based on Extracted Text

In [24]:
# Example Query 1 based on extracted text
if qa_chain and processed_documents:
    # Look for content related to the first HWP file (한영대학)
    hwp1_doc = next((doc for doc in processed_documents if '한영대학' in doc.metadata.get('파일명', '')), None)
    if hwp1_doc:
        # Formulate a query based on likely content (e.g., project purpose, scope, or timeline)
        query = "한영대학교 트랙운영 학사정보시스템 고도화 사업의 추진 배경 및 필요성은 무엇인가요?"
        response, query_time = time_rag_query(qa_chain, query) # Use the timing function

        print("Query:")
        print(query)
        print("\nGenerated Response:")
        print(response['result'])
        print(f"\nQuery Time: {query_time:.2f} seconds") # Print the execution time
        print("\nSource Documents:")
        for doc in response['source_documents']:
            print(f"--- Source: {doc.metadata.get('source', 'N/A')} (Chunk Index: {doc.metadata.get('chunk_index', 'N/A')}) ---")
            print(doc.page_content[:300] + "...") # Print a snippet of the source chunk
    else:
        print("Could not find processed document for 한영대학 to generate query.")
else:
    print("RAG chain or processed documents not initialized.")

Query:
한영대학교 트랙운영 학사정보시스템 고도화 사업의 추진 배경 및 필요성은 무엇인가요?

Generated Response:
한영대학교 트랙운영 학사정보시스템 고도화 사업의 추진 배경 및 필요성은 다음과 같습니다:

1. **교육 환경의 급격한 변화**: 인구 감소, 코로나19 팬데믹으로 인한 교육 환경 변화 및 인공지능의 활용 등으로 대학이 직면한 사회-기술 변화에 대응하기 위함입니다.

2. **분산된 시스템 및 데이터 통합 요구**: 노후화된 학사 시스템으로 인해 업무마다 분산된 정보가 존재하며, 시스템 통합에 대한 요구가 증가하고 있습니다.

3. **데이터 기반 대학 경영 지원 개선**: 정보 시스템 간의 연계 부족으로 경영 현황 파악 및 의사결정에 한계가 발생하고 있습니다.

4. **사용자 정보 서비스 접근성 개선**: 교내 구성원에게 제공되는 정보 서비스의 접근성과 유용성을 개선할 필요성이 커지고 있습니다.

5. **대학 경쟁력 강화**: 노후화된 시스템을 개선하여 정보 서비스의 품질을 높이고, 대학 교육 시스템의 경쟁력을 확보하기 위한 목적입니다.

Query Time: 9.74 seconds

Source Documents:
--- Source: /content/drive/MyDrive/Data/files/한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp (Chunk Index: N/A) ---
<표>





          


2024. 10.




<그림>

 

<표>






<그림>


<표>



<표>


□ 사 업 명 : 한영대학교 특성화 맞춤형 교육환경 구축 - 
             트랙운영 학사정보시스템 고도화
□ 사업예산 : 130,000,000원 범위 내 (VAT 포함)
□ 사업기간 : 계약일로부터 3개월 (안정화기간 1개월 포함)
  * 기간 및 일정은 학교 사정과 용역대상자와의 협의에 따라 조정될 수 있음
□ 입찰방법 : 제한경쟁입찰(협상에 의한 계약 체결)



<표

In [25]:
# Example Query 2 based on extracted text
if qa_chain and processed_documents:
    # Look for content related to the second HWP file (한국연구재단)
    hwp2_doc = next((doc for doc in processed_documents if '한국연구재단' in doc.metadata.get('파일명', '')), None)
    if hwp2_doc:
         # Formulate a query based on likely content (e.g., system function, users, or timeline)
        query = "한국연구재단의 대학산학협력활동 실태조사 시스템 기능개선 사업의 목표는 무엇인가요?"
        response, query_time = time_rag_query(qa_chain, query) # Use the timing function

        print("Query:")
        print(query)
        print("\nGenerated Response:")
        print(response['result'])
        print(f"\nQuery Time: {query_time:.2f} seconds") # Print the execution time
        print("\nSource Documents:")
        for doc in response['source_documents']:
            print(f"--- Source: {doc.metadata.get('source', 'N/A')} (Chunk Index: {doc.metadata.get('chunk_index', 'N/A')}) ---")
            print(doc.page_content[:300] + "...") # Print a snippet of the source chunk
    else:
        print("Could not find processed document for 한국연구재단 to generate query.")
else:
    print("RAG chain or processed documents not initialized.")

Query:
한국연구재단의 대학산학협력활동 실태조사 시스템 기능개선 사업의 목표는 무엇인가요?

Generated Response:
한국연구재단의 대학산학협력활동 실태조사 시스템 기능개선 사업의 목표는 UICC 시스템의 효율적인 관리와 운영을 지원하고, 사용자 요구를 반영하여 사용자 편의성을 강화하며, 실태조사 항목 지침 변경사항에 신속하고 정확하게 대응하는 것입니다.

Query Time: 6.69 seconds

Source Documents:
--- Source: /content/drive/MyDrive/Data/files/한국연구재단_2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선.hwp (Chunk Index: N/A) ---
<표>



<그림>

<표>







2024.  10.


<표>



<그림>


<표>


<그림>



<표>


<표>

1. 사업명: 2024년 대학 산학협력활동 실태조사 시스템(UICC) 기능개선
2. 추진배경 및 필요성
 □ 한국연구재단법 제5조, 산업교육진흥 및 산학연협력촉진에 관한 법률 제39조의2 및 제43조에 따라 실시하는 대학 산학협력활동 실태조사를 안정적으로 운영 필요 
 □ 항목 지침 변경사항 등 변화에 신속하고 정확하게 대응하며, 기능개선에 대한 사용자 요구를 반영하여 사용자 편의성 강화 필요
 □...
--- Source: /content/drive/MyDrive/Data/files/고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf (Chunk Index: N/A) ---
고려대학교 ROa UIVER SITY 설명 이뉴가 되는 업무의 재설계 및 정보화 밤만 제시{BPR 업무 프로서스 재설계 [상 업무는 ISP어서 탁사점점 전반어 걷쳐 도출권 업무 프로세스 재설계 대상문 기준으로 하머 주요 대상 업무는 아래와 같음 학과 흩업기준 R건과 정보화 정도 분석 및 차세대 정보화 개선 방안 대학원번 입시-학적생섬 시점에 따른 대학원생 정보시스템 사용 이수 및 개신 방만 도츠 {대학원 

In [26]:
# Example Query 3 based on extracted text
if qa_chain and processed_documents:
    # Look for content related to the third HWP file (한국생산기술연구원)
    hwp3_doc = next((doc for doc in processed_documents if '한국생산기술연구원' in doc.metadata.get('파일명', '')), None)
    if hwp3_doc:
        # Formulate a query based on likely content (e.g., system overview, purpose, or scope)
        query = "한국생산기술연구원의 EIP3.0 고압가스 안전관리 시스템 구축 용역의 사업 개요를 설명해 주세요."
        response, query_time = time_rag_query(qa_chain, query) # Use the timing function

        print("Query:")
        print(query)
        print("\nGenerated Response:")
        print(response['result'])
        print(f"\nQuery Time: {query_time:.2f} seconds") # Print the execution time
        print("\nSource Documents:")
        for doc in response['source_documents']:
            print(f"--- Source: {doc.metadata.get('source', 'N/A')} (Chunk Index: {doc.metadata.get('chunk_index', 'N/A')}) ---")
            print(doc.page_content[:300] + "...") # Print a snippet of the source chunk
    else:
        print("Could not find processed document for 한국생산기술연구원 to generate query.")
else:
    print("RAG chain or processed documents not initialized.")

Query:
한국생산기술연구원의 EIP3.0 고압가스 안전관리 시스템 구축 용역의 사업 개요를 설명해 주세요.

Generated Response:
EIP3.0 고압가스 안전관리 시스템 구축 용역은 다음과 같은 사업 개요를 가지고 있습니다.

1. **추진배경 및 필요성**:
   - **안전사고 증가**: 고압가스 취급 부주의로 인한 연구실 사고가 빈번하게 발생하고 있으며, 이는 인명 피해와 연구 기기 손상을 초래하고 있습니다.
   - **법적 요구사항**: 안전 관리에 대한 법적 요구사항과 규제가 강화되고 있어, 이를 준수하지 않을 경우 법적 책임과 벌금이 부과됩니다.
   - **연구 환경 보호**: 연구원들의 건강과 안전을 보호하고, 안전한 연구 환경을 제공하기 위해 체계적인 관리 시스템 구축이 필요합니다.

2. **사업 범위**:
   - 원내에서 관리하는 화학물질 운영현황의 통계 정보를 제공하는 시스템 구축.
   - 고압가스 화학물질의 정보와 판매허가 업체를 관리하기 위한 시스템 구축.
   - 고압가스 화학물질의 구매부터 안전성 검토 과정을 진행하기 위한 시스템 고도화.
   - 연구실과 법적 준수 사항을 관리하기 위한 시스템 고도화.
   - 화학제품 폐기·회수를 관리하기 위한 시스템 고도화.

3. **기대효과**:
   - **연구실 안전 확보**: 고압가스의 안전한 관리를 통해 연구원들의 생명과 신체를 보호하고, 연구실 내 안전사고를 예방하는 것을 목표로 합니다.
   - **법적 준수**: 정부 및 관련 기관의 규제와 법적 요구사항을 준수하여 법적 문제를 예방하고, 안전 관리 수준을 법적 기준 이상으로 유지합니다.
   - **효율적 관리 시스템 도입**: 고압가스의 사용, 저장, 폐기 등을 체계적으로 관리할 수 있는 시스템을 도입하여 효율성을 높이고, 안전 관리를 용이하게 합니다.

Query Time: 12.43 seconds

Source Documents:
--- Source: /content/drive/MyDrive/Data

In [27]:
# Example Query 4 based on extracted text
if qa_chain and processed_documents:
    # Look for content related to the fourth HWP file (인천광역시)
    hwp4_doc = next((doc for doc in processed_documents if '인천광역시' in doc.metadata.get('파일명', '')), None)
    if hwp4_doc:
        # Formulate a query based on likely content (e.g., system purpose, users, or key features)
        query = "인천광역시 도시계획위원회 통합관리시스템 구축 용역의 용역 개요는 무엇인가요?"
        response, query_time = time_rag_query(qa_chain, query) # Use the timing function

        print("Query:")
        print(query)
        print("\nGenerated Response:")
        print(response['result'])
        print(f"\nQuery Time: {query_time:.2f} seconds") # Print the execution time
        print("\nSource Documents:")
        for doc in response['source_documents']:
            print(f"--- Source: {doc.metadata.get('source', 'N/A')} (Chunk Index: {doc.metadata.get('chunk_index', 'N/A')}) ---")
            print(doc.page_content[:300] + "...") # Print a snippet of the source chunk
    else:
        print("Could not find processed document for 인천광역시 to generate query.")
else:
    print("RAG chain or processed documents not initialized.")

Query:
인천광역시 도시계획위원회 통합관리시스템 구축 용역의 용역 개요는 무엇인가요?

Generated Response:
인천광역시 도시계획위원회 통합관리시스템 구축 용역의 용역 개요는 다음과 같습니다:
- 용역명: 도시계획위원회 통합관리시스템 구축 용역
- 기간: 착수일로부터 180일
- 사업비: 금150,000,000원 (VAT 포함)
- 계약방법: 제한경쟁입찰(협상에 의한 계약)

Query Time: 5.98 seconds

Source Documents:
--- Source: /content/drive/MyDrive/Data/files/인천광역시_도시계획위원회 통합관리시스템 구축용역.hwp (Chunk Index: N/A) ---
<그림>










<표>



<표>


<표>


<표>


 1. 과 업 명: 「도시계획위원회 통합관리시스템 구축 용역」

 2. 용역개요
  ○ 용 역 명: 도시계획위원회 통합관리시스템 구축 용역
  ○ 기    간: 착수일로부터 180일
  ○ 사 업 비: 금150,000,000원 (VAT 포함)
  ○ 계약방법: 제한경쟁입찰(협상에 의한 계약) 
  
<표>

  
<표>

  
<표>


 3. 과업배경 및 목적
  ○ (수작업에 의한 업무의 복잡성) 여러 단계의 위원회 운영업무를 수작업으로 처리, 업무의 복잡성...
--- Source: /content/drive/MyDrive/Data/files/고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf (Chunk Index: N/A) ---
고려대학교 ROa Mtfr cT 역량기반 교과욕 운영체계 정보화 개선 감의계학서 상 역량항묵 입력스 서움캠퍼스 역량으로만 구축하여 세증캠퍼스 적용 불가 2024년부터 문명되는 CQI보고서지 현재는 역량관리시스템 KUchive KUSEUM 에서 관리하고 있어 학사행정시스템으로 기능 이관 필요 학상들의 수감소감 및 그에 대한 사후조치튿 현재는 역실로 관리 "교마 비교로 #품 재광 마 | 싶물?-펴화 'n질a하 팬급

In [28]:
# Example Query 5 based on extracted text
if qa_chain and processed_documents:
    # Look for content related to the PDF file (고려대학교)
    pdf_doc = next((doc for doc in processed_documents if '고려대학교' in doc.metadata.get('파일명', '')), None)
    if pdf_doc:
        # Formulate a query based on likely content (e.g., project background, scope, or goals)
        query = "고려대학교 차세대 포털·학사 정보시스템 구축 사업의 추진 배경을 설명해주세요."
        response, query_time = time_rag_query(qa_chain, query) # Use the timing function

        print("Query:")
        print(query)
        print("\nGenerated Response:")
        print(response['result'])
        print(f"\nQuery Time: {query_time:.2f} seconds") # Print the execution time
        print("\nSource Documents:")
        for doc in response['source_documents']:
            print(f"--- Source: {doc.metadata.get('source', 'N/A')} (Chunk Index: {doc.metadata.get('chunk_index', 'N/A')}) ---")
            print(doc.page_content[:300] + "...") # Print a snippet of the source chunk
    else:
        print("Could not find processed document for 고려대학교 to generate query.")
else:
    print("RAG chain or processed documents not initialized.")

Query:
고려대학교 차세대 포털·학사 정보시스템 구축 사업의 추진 배경을 설명해주세요.

Generated Response:
고려대학교 차세대 포털·학사 정보시스템 구축 사업의 추진 배경은 다음과 같습니다:

1. **교육 환경의 급격한 변화**: 인구 감소와 코로나19 팬데믹으로 인해 교육 환경이 변화하고 있으며, 인공지능의 활용 등으로 대학이 사회-기술 변화에 직면하고 있습니다.

2. **분산된 시스템 및 데이터 통합 요구**: 급변하는 교육 환경 속에서 정보화 요구가 증가하고 있으나, 기존의 노후화된 학사 시스템이 업무마다 분산되어 있어 시스템 통합에 대한 필요성이 커지고 있습니다.

3. **데이터 기반 대학 경영 지원 개선**: 분절된 부서 시스템 간 정보 연계 미비와 정보화 표준 부재로 인해 경영 현황 파악 및 의사결정에 한계가 발생하고 있습니다.

4. **사용자 정보 서비스 접근성 개선**: 교내 구성원에게 제공되는 포털 서비스의 정보 접근성과 유용성에 대한 개선 요구가 상당히 높습니다.

5. **대학 경쟁력 강화 및 전략 목표 달성 지원**: 노후화된 시스템의 차세대 구축을 통해 정보 서비스 품질을 강화하고, 대학의 교육 시스템 경쟁력을 확보하고자 하는 목표가 있습니다.

Query Time: 9.73 seconds

Source Documents:
--- Source: /content/drive/MyDrive/Data/files/고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf (Chunk Index: N/A) ---
고려대학교 ROa UIVER SITY 제 안 요 청 서 고려대학교 차세대 포털 학사 정보시스템 구축 사업 2024. 7. 07 aE 고려대학교 KOREA UNIVERSITY 본 자고; 게민내공의 신편움 취한 "포자고로 부거으로 '년부지 사용타;  형위: 구립-
고려대학교 ROa UIVER SITY 목   차 사업개요 1. 사업 개요 2 사업 배경 3. 사업 범위 :. 기대 증과 I. 현황 및 문제점 1. 일반 된참 

# Task
Build an advanced RAG system using Langchain to answer questions based on 4 HWP files and 1 PDF file from "/content/drive/MyDrive/Data/files". The system should:
1. Load and process the specified HWP (using pyhwp or hwp5txt/hwp5html) and PDF (using easyocr) files.
2. Integrate metadata from "data_list.csv" with the documents.
3. Implement semantic chunking.
4. Utilize metadata filtering, increased top-k, MMR, and multi-query for improved retrieval.
5. Employ prompt engineering, adjust generation parameters (top-p, max_tokens), control answering tone/style, and maintain conversational context for enhanced generation.
6. Evaluate the system's performance using new example queries derived from the content of the selected files.

# Task
Implement metadata filtering and conversational memory in the RAG chain and update the example queries to demonstrate these features.

## Implement metadata filtering

### Subtask:
Modify the retrieval process to allow filtering documents based on metadata.


**Reasoning**:
Update the RetrievalQA chain creation to include metadata filtering capabilities in the retriever.



In [59]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_openai import OpenAIEmbeddings
from langchain.retrievers import MultiQueryRetriever
from langchain.prompts import PromptTemplate
from google.colab import userdata

try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except userdata.SecretNotFoundError:
    print("Error: OPENAI_API_KEY not found in Colab secrets.")
    print("Please add your OpenAI API key to the Colab secrets manager (🔑 icon on the left) with the name 'OPENAI_API_KEY'.")
    OPENAI_API_KEY = None

if OPENAI_API_KEY and vectorstore:
    llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY, temperature=0.3, max_tokens=1500, top_p=0.8)

    template = """다음 문서를 사용하여 질문에 답하세요.
    답변은 가능한 한 정확하고 간결하게 작성하세요.
    제공된 문서에 없는 내용은 언급하지 마세요.
    문서:
    {context}

    질문: {question}

    답변:"""
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

    # Update the base retriever to accept filters
    # The filter structure will be determined dynamically based on the query in later steps.
    # For now, we just modify the retriever to potentially use a filter.
    # Langchain's FAISS retriever supports filtering by metadata.
    # The filter needs to be a dictionary specifying the key and value to filter on.
    # Example: {"발주 기관": "한영대학"}
    # We will demonstrate filtering in the example queries in the next step.

    # We will create a base retriever that accepts a filter parameter in its get_relevant_documents method
    # However, the standard .as_retriever() method from FAISS doesn't directly expose a filter parameter
    # in this way for simple use with RetrievalQA.
    # A more direct way to integrate filtering with RetrievalQA is to pass the filter
    # in the query to the chain. Let's revert the retriever creation back to how it was,
    # and the filtering logic will be applied when invoking the QA chain later.

    # Reverting to the multi-query retriever without explicit filter in the retriever definition
    base_retriever = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 10, "fetch_k": 20}
    )

    multiquery_retriever = MultiQueryRetriever.from_llm(
        retriever=base_retriever,
        llm=llm
    )

    final_retriever = multiquery_retriever

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=final_retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
    )

    print("RetrievalQA chain created, ready for metadata filtering in query invocation.")

else:
    print("Skipping RAG chain creation due to missing API key or vector store.")
    qa_chain = None

RetrievalQA chain created, ready for metadata filtering in query invocation.


In [60]:
import time

def time_rag_query_with_filter_corrected(qa_chain, query, metadata_filter=None):
    """
    Times the execution of a RAG query with optional metadata filtering,
    correctly applying the filter to the retriever's search_kwargs.

    Args:
        qa_chain: The RAG chain object.
        query (str): The query string.
        metadata_filter (dict, optional): A dictionary specifying metadata filter criteria.
                                          Defaults to None.

    Returns:
        tuple: A tuple containing the response dictionary and the execution time in seconds.
    """
    start_time = time.time()

    # Access the retriever within the QA chain
    # The exact path might vary slightly depending on the chain type and retriever wrapping
    # If using MultiQueryRetriever wrapped around a base retriever, we need to access the base retriever
    if hasattr(qa_chain.retriever, 'retrievers') and isinstance(qa_chain.retriever.retrievers, list):
         # This case might apply if using something like EnsembleRetriever or similar multi-retriever setups
         # For MultiQueryRetriever, the base retriever is usually accessed differently
         base_retriever = qa_chain.retriever.retrievers[0] # Assuming the first retriever is the base one
    elif hasattr(qa_chain.retriever, 'retriever'):
         # This is the case for ContextualCompressionRetriever and likely MultiQueryRetriever
         base_retriever = qa_chain.retriever.retriever
    else:
         # Direct retriever instance
         base_retriever = qa_chain.retriever


    # Temporarily modify the retriever's search_kwargs to include the filter
    original_search_kwargs = base_retriever.search_kwargs
    if metadata_filter:
        # Create a copy of original search_kwargs and add the filter
        modified_search_kwargs = original_search_kwargs.copy()
        modified_search_kwargs["filter"] = metadata_filter
        base_retriever.search_kwargs = modified_search_kwargs
        print(f"Applying filter: {metadata_filter}")
    else:
        # Ensure no filter is applied if metadata_filter is None
        if "filter" in original_search_kwargs:
             modified_search_kwargs = original_search_kwargs.copy()
             del modified_search_kwargs["filter"]
             base_retriever.search_kwargs = modified_search_kwargs
        print("No filter applied.")


    # Invoke the chain with the query
    # The search_kwargs are now set on the retriever itself
    response = qa_chain.invoke({"query": query})

    # Restore the original search_kwargs on the retriever
    base_retriever.search_kwargs = original_search_kwargs

    end_time = time.time()
    execution_time = end_time - start_time
    return response, execution_time

# Example Query 1 with Metadata Filter (Filter by '발주 기관': '한영대학')
if qa_chain and processed_documents:
    query = "이 사업의 추진 배경 및 필요성은 무엇인가요?"
    metadata_filter = {"발주 기관": "한영대학"}
    print(f"Query: {query}")
    response, query_time = time_rag_query_with_filter_corrected(qa_chain, query, metadata_filter)

    print("\nGenerated Response:")
    print(response['result'])
    print(f"\nQuery Time: {query_time:.2f} seconds")
    print("\nSource Documents:")
    for doc in response['source_documents']:
        print(f"--- Source: {doc.metadata.get('source', 'N/A')} ---")
        print(f"Metadata: {doc.metadata}")
        print(doc.page_content[:300] + "...")
else:
    print("RAG chain or processed documents not initialized.")

print("-" * 50)

# Example Query 2 with Metadata Filter (Filter by '파일형식': 'pdf')
if qa_chain and processed_documents:
    query = "이 사업의 추진 배경을 설명해주세요."
    metadata_filter = {"파일형식": "pdf"}
    print(f"Query: {query}")
    response, query_time = time_rag_query_with_filter_corrected(qa_chain, query, metadata_filter)

    print("\nGenerated Response:")
    print(response['result'])
    print(f"\nQuery Time: {query_time:.2f} seconds")
    print("\nSource Documents:")
    for doc in response['source_documents']:
        print(f"--- Source: {doc.metadata.get('source', 'N/A')} ---")
        print(f"Metadata: {doc.metadata}")
        print(doc.page_content[:300] + "...")
else:
    print("RAG chain or processed documents not initialized.")

print("-" * 50)

# Example Query 3 with Metadata Filter (Filter by '사업명')
if qa_chain and processed_documents:
    # Find an example 사업명 from the processed documents
    example_doc = processed_documents[0] if processed_documents else None
    if example_doc and '사업명' in example_doc.metadata:
        example_사업명 = example_doc.metadata['사업명']
        query = f"'{example_사업명}' 사업의 개요는 무엇인가요?"
        metadata_filter = {"사업명": example_사업명}
        print(f"Query: {query}")
        response, query_time = time_rag_query_with_filter_corrected(qa_chain, query, metadata_filter)

        print("\nGenerated Response:")
        print(response['result'])
        print(f"\nQuery Time: {query_time:.2f} seconds")
        print("\nSource Documents:")
        for doc in response['source_documents']:
            print(f"--- Source: {doc.metadata.get('source', 'N/A')} ---")
            print(f"Metadata: {doc.metadata}")
            print(doc.page_content[:300] + "...")
    else:
         print("Could not find an example document with '사업명' metadata for Query 3.")
else:
    print("RAG chain or processed documents not initialized.")

Query: 이 사업의 추진 배경 및 필요성은 무엇인가요?
Applying filter: {'발주 기관': '한영대학'}

Generated Response:
이 사업의 추진 배경 및 필요성은 다음과 같습니다:

- 학사제도 개편과 연계하여 전공교과목 선택폭을 넓히고, 트랙제 교육과정 참여자에게 다양한 진로선택의 기회를 제공하여 취업문을 확대하기 위함입니다.
- 트랙제 교육과정의 도입 및 운영을 통해 산업현장의 경쟁력을 강화하고, 기업 수요에 맞춘 교육과정 운영 및 활성화를 통해 교육과정의 내실화를 도모하고자 합니다.
- 산업체 및 지역사회와 연계하여 현장실무형 인재를 양성하기 위한 체계적인 지원이 필요합니다.
- 트랙기반 교육과정의 운영 및 관리 체계를 효과적으로 지원하고, 교수자·학습자 중심의 교육환경을 조성하여 대학 교육의 가치를 구현하기 위한 시스템이 요구됩니다.

Query Time: 7.00 seconds

Source Documents:
--- Source: /content/drive/MyDrive/Data/files/한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp ---
Metadata: {'source': '/content/drive/MyDrive/Data/files/한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp', '공고 번호': '20241001798', '공고 차수': 0.0, '사업명': '한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보시스템 고도화', '사업 금액': 130000000.0, '발주 기관': '한영대학', '공개 일자': '2024-10-04 13:51:23', '입찰 참여 시작일': nan, '입찰 참여 마감일': '2024-10-15 17:00:00', '사업 요약': '- 한영대학교 특성화 맞춤형 교육환경 구축을 위해 트랙운영 학사정보시스템을 고도화한다.\n- 트랙제도를 도입하여 다양한 진로 선택 기회를 제공하고 산업현장의 경쟁력을 강화한다.\n

## Add conversational memory

### Subtask:
Integrate conversational memory into the RAG chain to maintain context across turns.


**Reasoning**:
Integrate conversational memory into the RAG chain to maintain context across turns.



In [61]:
from langchain.chains import RetrievalQA
# Import the necessary components for conversational memory
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain.retrievers import MultiQueryRetriever
from langchain.prompts import PromptTemplate
from google.colab import userdata

try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except userdata.SecretNotFoundError:
    print("Error: OPENAI_API_KEY not found in Colab secrets.")
    print("Please add your OpenAI API key to the Colab secrets manager (🔑 icon on the left) with the name 'OPENAI_API_KEY'.")
    OPENAI_API_KEY = None # Set to None to prevent further errors

if OPENAI_API_KEY and vectorstore:
    # Initialize the ChatOpenAI model for generation (will also be used for Multi-Query)
    llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=OPENAI_API_KEY, temperature=0.3, max_tokens=1500, top_p=0.8)

    # Define a refined prompt template that includes chat history
    # This template guides the language model on how to use the retrieved context and the conversation history
    template = """다음 채팅 기록과 문서를 사용하여 질문에 답하세요.
    채팅 기록:
    {chat_history}

    문서:
    {context}

    질문: {question}

    답변:"""
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

    # Base Retriever with higher k and MMR
    base_retriever = vectorstore.as_retriever(
        search_type="mmr", # Use MMR search
        search_kwargs={"k": 10, "fetch_k": 20} # Increased k and fetch_k
    )

    # Implement Multi-Query Retriever
    # The multi-query retriever itself does not inherently handle conversational context
    # It generates multiple queries based on the *current* query.
    # Conversational memory needs to be integrated at the chain level.
    multiquery_retriever = MultiQueryRetriever.from_llm(
        retriever=base_retriever,
        llm=llm
    )

    final_retriever = multiquery_retriever

    # Initialize ConversationBufferMemory to store chat history
    # memory_key='chat_history' corresponds to the chat history variable in the prompt template
    # input_key='question' corresponds to the variable in the chain that will receive the user's query
    memory = ConversationBufferMemory(memory_key="chat_history", input_key="question")


    # Create a RetrievalQA chain with the advanced retriever and refined prompt
    # Add the memory parameter to the chain
    qa_chain_with_memory = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff", # 'stuff' chain type is suitable for including context in the prompt
        retriever=final_retriever, # Use the advanced retriever
        return_source_documents=True,
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT, "memory": memory} # Pass the refined prompt and memory
    )

    # Update the qa_chain variable
    qa_chain = qa_chain_with_memory

    print("Advanced RetrievalQA chain created with Multi-Query, MMR, refined generation parameters/prompt, and conversational memory.")

else:
    print("Skipping RAG chain creation due to missing API key or vector store.")
    qa_chain = None # Set qa_chain to None if creation failed

Advanced RetrievalQA chain created with Multi-Query, MMR, refined generation parameters/prompt, and conversational memory.


**Reasoning**:
Demonstrate the conversational memory by asking follow-up questions that rely on the previous turn's context.



In [62]:
# Function to time RAG query (re-using the previous definition for consistency)
import time

def time_rag_query(qa_chain, query):
    """
    Times the execution of a RAG query.

    Args:
        qa_chain: The RAG chain object.
        query (str): The query string.

    Returns:
        tuple: A tuple containing the response dictionary and the execution time in seconds.
    """
    start_time = time.time()
    response = qa_chain.invoke({"query": query})
    end_time = time.time()
    execution_time = end_time - start_time
    return response, execution_time

# Example Conversation with Conversational Memory
if qa_chain and processed_documents:
    print("--- Starting Conversation with Memory ---")

    # First turn: Ask about a specific project's background (similar to previous examples)
    query1 = "한영대학교 트랙운영 학사정보시스템 고도화 사업의 추진 배경 및 필요성은 무엇인가요?"
    print(f"\nUser Query 1: {query1}")
    response1, time1 = time_rag_query(qa_chain, query1)
    print("\nGenerated Response 1:")
    print(response1['result'])
    print(f"Query 1 Time: {time1:.2f} seconds")
    # Source documents are returned but not explicitly printed here for brevity,
    # as the focus is on the conversational flow.

    print("-" * 30)

    # Second turn: Ask a follow-up question that relies on the previous turn's context
    # This query doesn't mention "한영대학교" or the full project name,
    # relying on the memory to understand "이 사업" (this project).
    query2 = "그 사업의 사업 기간은 얼마나 되나요?"
    print(f"\nUser Query 2: {query2}")
    response2, time2 = time_rag_query(qa_chain, query2)
    print("\nGenerated Response 2:")
    print(response2['result'])
    print(f"Query 2 Time: {time2:.2f} seconds")

    print("-" * 30)

    # Third turn: Another follow-up question
    query3 = "사업 금액은 얼마인가요?"
    print(f"\nUser Query 3: {query3}")
    response3, time3 = time_rag_query(qa_chain, query3)
    print("\nGenerated Response 3:")
    print(response3['result'])
    print(f"Query 3 Time: {time3:.2f} seconds")

    print("--- Conversation with Memory Ended ---")

else:
    print("RAG chain with memory or processed documents not initialized.")

--- Starting Conversation with Memory ---

User Query 1: 한영대학교 트랙운영 학사정보시스템 고도화 사업의 추진 배경 및 필요성은 무엇인가요?

Generated Response 1:
한영대학교 트랙운영 학사정보시스템 고도화 사업의 추진 배경 및 필요성은 다음과 같습니다:

1. **학사제도 개편**: 학사제도 개편과 연계하여 전공교과목 선택폭을 넓히고, 트랙제 교육과정 참여자에게 다양한 진로선택의 기회를 제공하여 취업문을 확대할 필요성이 있습니다.

2. **산업체 수요 맞춤 교육**: 산업체의 수요에 맞춘 교육과정 운영 및 활성화를 통해 교육과정의 내실화를 도모하고, 산업현장의 경쟁력을 강화해야 합니다.

3. **정보화 요구 증가**: 급변하는 교육환경 속에서 정보화 요구가 증가하고 있으며, 기존의 노후화된 학사 시스템으로 인해 업무가 분산되어 있어 시스템 통합에 대한 요구가 커지고 있습니다.

4. **데이터 기반 의사결정 지원**: 분절된 시스템 간 정보 연계 및 정보화 표준 부재로 인해 경영현황 파악 및 의사결정 시 한계가 발생하고 있습니다. 따라서, 데이터 기반의 경영정보 제공이 필요합니다.

5. **사용자 정보 서비스 접근성 개선**: 교내 구성원의 정보 접근성을 높이기 위한 포털 서비스의 제공이 필요하며, 이를 통해 사용자 편의성을 향상시킬 수 있습니다.

6. **대학 경쟁력 강화**: 노후화된 시스템의 차세대 구축을 통해 정보 서비스의 품질을 강화하고, 대학의 교육 시스템 경쟁력을 확보해야 합니다.

이러한 배경과 필요성에 따라 한영대학교는 트랙운영 학사정보시스템의 고도화를 추진하고 있습니다.
Query 1 Time: 11.31 seconds
------------------------------

User Query 2: 그 사업의 사업 기간은 얼마나 되나요?

Generated Response 2:
한영대학교 트랙운영 학사정보시스템 고도화 사업의 사업 기간은 계약일로부터 24개월 이내입니다.
